# CI CD with AWS

### Introduction

In this lesson, we'll see how we can begin use github actions to redeploy our code on AWS.  And we'll do so with terraform.  Let's get started.

### Creating a Role

Now the way that we'll give Github the ability to interact with our AWS services is to create a Role.  As we know, a role is like a keycard that has permissions -- and we can give this keycard to AWS resources like an EC2 machine.  It turns out that we can also allow a Github repository to assume an AWS role.  And that's how we can give our repository the ability to interact with our AWS resources.

In this next section we'll see how we can do this.

#### 1. Creating an Identity Provider

Ok, so because we want to allow our Github account to access an AWS role, the first step is to create an identity provider.  An identity provider is an AWS service that establishes trust between your AWS account and an external identity system -- here the login for Github. 

Ok, so to create an identity provider, login to AWS and go to `IAM` > `Identity Providers`.

<img src="./identity-click.png" width="30%">

From there click the orange button `Add Provider`, and then select `OpenID Connect`.

<img src="./openid-connect.png" width="60%">

At this point, you'll be asked to add a few pieces of information.

2. Ok, so next we specify the identity provider and the Audience.  Fill in the following values.

* Provider Url: `https://token.actions.githubusercontent.com`
* Audience: `sts.amazonaws.com`

> This information is to allow AWS to authenticate Github to use it's service.  It will need to make an API request to Github's tokens url (the provider url), and then github will need to provide back information to the AWS secure token service (sts), to get the token to login.

* Click get thumbprint.

And then click `Add Provider`.

#### 2. Creating a new role


Ok, so we just created a provider, and that provider tells AWS how to interact with Github -- where to request authentication information, and how Github should send back that information to AWS.  Next up is to create a role that will use that identity provider to grant access to our Github.

* Go to `Access Management > Roles > Create Role`


<img src="./actions-deploy.png" width="60%">

1. Then among the panels above, you can select `Web Identity`. 

Notice the description.  It says that Web Identity allows users *federated* by the external web identity provider to assume this role. 

By federated, it means that an external user (eg. github) can assume a role in this account.  

> So just like in government federated means different localities operating under a centralized system, here we are allowing different external providers to operate under our single AWS account, as opposed to each getting their own AWS account. 

2. Select the Identity Provider and Audience
* For Identity Provider, select the `token.actions.githubusercontent.com`
* For Audience, select `sts.amazonaws.com`

3. From there, specify:
    * **Your** github account, and
    * The github repository you want to integrate your github actions with.

4. Then click `Add Provider`

From there, we'll need to grant a certain level of access.  For now, we can select `Admin Access`.

> Once this is working, it's a good idea to pair down the access to just what is needed (here mainly ECR and EC2 access).  This ie because you the **principle of least priviledge** says that you want to grant as little access to others (almost like everyone is on a need to know basis).

After clicking `Add Provider`, you'll see that the following is generated Assume Role policy was generated (just with your github account and repo information).

```bash
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Principal": {
                "Federated": "arn:aws:iam::****:oidc-provider/token.actions.githubusercontent.com"
            },
            "Condition": {
                "StringEquals": {
                    "token.actions.githubusercontent.com:aud": [
                        "sts.amazonaws.com"
                    ]
                },
                "StringLike": {
                    "token.actions.githubusercontent.com:sub": [
                        "repo:jigsawlabs-student/actions-deploy:*"
                    ]
                }
            }
        }
    ]
}
```

Let's take a moment to understand the policy generated above:

* The action is to allow githubusercontent the action of `sts:AssumeRoleWithWebIdentity`.  So this means that the secure token service can submit a token to github, granting the ability to assume the role we created.

* The Principal specifies only github can assume this role.
* The last `StringEquals` Condition confirms that the audience Github Actions is sending it's request to, is to the `sts` (secure token service).
* The `StringLike`: This ensures the github token is issued for a specific token.  The `*` in `actions-deploy:*` means that any branch in the `actions-deploy` repository can request the token from AWS.

Then give the role a name, like `github-actions-deploy` and click `Create Role`.  

### Update Action

Ok, so we just created the role, and we know that github will make a request to assume that role.  Because AWS has an assume role policy that allows our account and repository to assume the role, our repository will (hopefully) be successful.

So what's next is we'll want our Github action to request to assume the role.  To set this up, we'll first need to find that role's ARN.  So click on `IAM` > `Roles`, and search for the role that you just created.

Then, after finding the role you just created, click on it, and to the right you'll see the ARN.  We'll use that next.

<img src="./arn-actions.png" width="50%">

We want to add our that role's ARN as a secret.  So to do that, go to your github repository for the action.  And click on `Settings` > `Secrets and variables`.

<img src="./settings-secrets-ci.png" width="30%">

From there, click on `New Repository Secret`.

<img src="./new-repo-secret.png" width="60%">

And give it a Name of `IAM_ROLE`, and a value of your AWS arn.

### Logging in with AWS

Ok, so next let's login with AWS.  If you look at the repository, you'll see the key step:

```bash
- name: Configure AWS credentials
      uses: aws-actions/configure-aws-credentials@v4
      with:
        role-to-assume: ${{ secrets.IAM_ROLE }}
        role-session-name: AWSSession
        aws-region: us-east-1
```

Here, we use the aws-actions/configure-aws-credentials step, defined [here](https://github.com/aws-actions/configure-aws-credentials).  And for the role to assume, we use our `IAM_ROLE` declared in SQL.  And then we add the `role-session-name` as AWSSession and the aws-region as our region (for me it's us-east-1).

The other step we need is adding the permission key up top:

```bash
permissions:
        id-token: write # This is required for requesting the JWT  
        contents: read
```

And if you care...

* `id-token`: `write` grants the workflow permission to request an OpenID Connect (OIDC) ID token.
The ID token is a JWT (JSON Web Token) that can be used to authenticate the GitHub Actions runner with external services that support OIDC (Open Identity), such as AWS.

* `contents: read` - This allows reading the contents of your github repository, as is performed in `actions/checkout`.  If you choose not put this in, your code should still work -- but [the documentation](https://docs.github.com/en/actions/deployment/security-hardening-your-deployments/configuring-openid-connect-in-amazon-web-services#adding-permissions-settings) recommends including, so let's do that.

You can see that for the last step, we see if we can use our credentials to log in to ECR.

Ok, that should be it.

* Try it out

So now let's try it out.  Navigate to the `web_app` folder, and push up this code to the repository you have set up in Github.  Then make a commit and push up the main branch.

Finally, click on `Actions`, and then your most recent workflow run.  If it turns green, and you have all checkmarks, then these steps worked properly.

<img src="./actions-initial.png" width="50%">

### Recap

In this lesson, we created an identity provider and a role.  One common question is, why do we need to build both?  The short answer is because the identity provider allows for *authentication* while the role is for *authorization*.

* Authentication - By authentication we mean logging you in.  In this case, the identity provider allows you to use your login to Github to login to AWS.  
* Authorization - The role says what you are authorized to do.  By authorization we mean, once we know who you are, what capabilities do you have.  Above, we created a role that has an assume role policy for our identity provider and that has admin access.

### Resources

[Github docs - AWS deployments](https://docs.github.com/en/actions/deployment/security-hardening-your-deployments/configuring-openid-connect-in-amazon-web-services#adding-the-identity-provider-to-aws)

[Skund Notes - Github Actions AWS](https://skundunotes.com/2023/02/28/securely-integrate-aws-credentials-with-github-actions-using-openid-connect/)

[Skund Notes - Github Actions TF](https://skundunotes.com/2023/03/07/ci-cd-with-terraform-and-github-actions-to-deploy-to-aws/)